Sources of help:
https://stackoverflow.com/questions/12477823/calling-a-function-in-a-separate-file-in-python
https://stackoverflow.com/questions/20186344/ipynb-import-another-ipynb-file

https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.html
https://stackoverflow.com/questions/19482970/get-list-from-pandas-dataframe-column-headers
https://stackoverflow.com/questions/19379120/how-to-read-a-config-file-using-python
https://stackoverflow.com/questions/12943819/how-to-prettyprint-a-json-file
https://stackoverflow.com/questions/42370977/how-to-save-a-new-sheet-in-an-existing-excel-file-using-pandas

In [11]:
#data science stuff
import numpy as np
import pandas as pd
import pprint
import configparser
config = configparser.ConfigParser()
config.read('config.INI')

#from ipynb.fs.full.readMaps import *
#import ipynb.fs.full.readMaps
%run readMaps.ipynb
%run readMPLinks.ipynb
%run readTeams.ipynb
%run createLeaderboard.ipynb
%run osuApiRequests.ipynb
%run processMP.ipynb

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

#requests
import requests
print("dab")

dab


In [20]:
#Get input
is_team_tourney = input("Is this a team tournament? (y/n) ")
is_team_tourney = True if is_team_tourney == 'y' else False

team_size = 0
if is_team_tourney == True:
    team_size = int(input("Team size? "))

#winning condition: score = 0, accuracy = 1, combo = 2, score v2 = 3
desired_scoring_type = input("What scoring? \
    (0)score, (1)acc, (2)combo, (3)scorev2" )
desired_scoring_type = str(desired_scoring_type)

#stage want to analyze
stage = input("What stage? ")

all_maps = readMaps('./Starter Sheets/Maps3.xlsx') #df of the maps and their mod in each stage
mp_ids = readMPLinks('./Starter Sheets/MPids.xlsx') #df of the mps in each stage


Is this a team tournament? (y/n) y
Team size? 2
What scoring?     (0)score, (1)acc, (2)combo, (3)scorev23
What stage? Group Stage
Group Stage


In [6]:
#read teams sheet if is team tournament
teams = 0
teamsInfo = {}
if is_team_tourney == True:
    teams = readTeams('./Starter Sheets/Teams.xlsx')

    
#labels = list(all_maps)
#stages = labels[0::2] #stages in the tournament
#stage_idx = 0



#---Process the stage of the tournament---
maps_in_stage_df = all_maps.loc[:, stage:stage + " Mods"] #df of map_ids and mod
map_ids_in_stage = maps_in_stage_df.iloc[:, 0].tolist()
map_ids_in_stage = [map_id for map_id in map_ids_in_stage if map_id != '-1']
map_mods_in_stage = maps_in_stage_df.iloc[:, 1].tolist()
map_mods_in_stage = [mod for mod in map_mods_in_stage if mod != '-1']

mp_ids_in_stage = mp_ids[stage] #list of mp_ids in stage

mods_for_each_map = dict(zip(map_ids_in_stage, \
                            map_mods_in_stage)) #track the mod each map uses
map_counts_in_stage = {} #track how many times each map was picked
for map_id in map_ids_in_stage:
    map_counts_in_stage[map_id] = 0

#Create empty leaderboards for all the maps in a stage and append to 
#all_leaderboards
leaderboards_in_stage = {} #dict of map_ids each having a leaderboard
for map_id in map_ids_in_stage:
    if map_id == '0': #@TODO why are there still '0's?
        continue
    leaderboards_in_stage[map_id] = createLeaderboard(map_id, is_team_tourney)

#Parse each match in this stage. On each match we: append to appropriate
#leaderboard, update teaminfo, update map_count dict

for mp_id in mp_ids_in_stage:
    #@TODO: add team_info functionality
    processMP(mp_id, leaderboards_in_stage, map_counts_in_stage, 
              is_team_tourney, teams, desired_scoring_type, 
              mods_for_each_map)  



#stage_idx += 2
pprint.pprint(leaderboards_in_stage)
print("-----------------------------------------------------------------")
    

#Construct the 'Overview', 'Best Plays', and 'Teams Overview' pages



     3rd err
     3rd err
     3rd err
     3rd err
     3rd err
     3rd err
     2nd err
     2nd err
     2nd err
     2nd err
     2nd err
     2nd err
     2nd err
     2nd err
     3rd err
     1st err: 3 45433386
     3rd err
     3rd err
     3rd err
     3rd err
{'113592': {'map_id': '113592',
            'scores':      #           Player                   Team Rank    Score Combo  Accuracy  \
0    1           sentez              Bulamadım    A   631337   829     98.17   
1    2            Entry              Bulamadım    S  1001259  1397     99.64   
2    3         NoVaLian                  Galer    A   392453   580     94.16   
3    4       Mr kamfret                  Galer    A   548207   870     96.98   
4    5     DwarfSpykerr       Cute Anime Girls    S   668218   946     98.38   
5    6          xHei_57       Cute Anime Girls    A   471141   576     97.65   
6    7       Mr kamfret                  Galer    A   528404   678     97.74   
7    8     -Nao Tomori-           

 '38581': {'map_id': '38581',
           'scores':        #           Player                   Team Rank    Score Combo  \
0      1          Cynomeg         seegii fanboys    S   827739   608   
1      2       Z3tsubouou         seegii fanboys    A   552635   460   
2      3    Spartan Plume                 uhh...    A   655561   416   
3      4        Gokateigo                 uhh...    S  1030563   689   
4      5           Shaper                           S  1044678   689   
5      6           Fretzi           Here for #1s    S   766630   531   
6      7       PainTensei        cute couple owo    A   463980   383   
7      8         loveleft        cute couple owo    S  1017267   689   
8      9            Barus        Futurus Sanguis    A   444777   313   
9     10       Kubunikusi        Futurus Sanguis    A   513839   323   
10    11           lokser           Andrzej Duda    S  1013572   689   
11    12         krazi225           Andrzej Duda    S   974782   688   
12    13     

In [21]:
#create dict of map ids to map name
map_ids_to_names = {}
for map_id in map_counts_in_stage:
    map_name = get_beatmaps(map_id)[0]['title']
    map_ids_to_names[map_id] = map_name


In [19]:
#create the local file containing all the leaderboards for stage
path = stage + '.xlsx'
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
for map_id in leaderboards_in_stage:
    leaderboards_in_stage[map_id]['scores'].to_excel(writer, sheet_name = map_id)
    
writer.save()
writer.close()

In [ ]:
#create the google spreadsheet containing all the leaderboards for stage


'CARTOON HEROES (Speedy Mix)'